# Image Stitching
This notebook contains the code that implements a simple stitching mechanism.
The idea is to find the corresponding points between the images and then after having defined an order between the images, stitch them one after the other. Thus, the first image is stitched with the second, the third is stitched with the result of the previous stitching, and on so.

## Importing libraries

In [1]:
import cv2 as cv
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import ipynb.fs.defs.Utils as Utils
from matplotlib.pyplot import figure

## Functions definition

In [2]:
#This function transform an image according to an homography
def transform_image(ref_index, #Index of the reference image
                    transf_index, #Index of the source image
                    image, #Image to be transformed in the reference frame
                    T_norm, #Normalization matrix
                    Z, #Matrix containing the homographies between the images
                    warp_shape = [2000,1000] #Dimension of the final image
                   ):
    
        #Retrieving the homography from the source image to the reference one
        h = Z[(ref_index)*3:(ref_index+1)*3,(transf_index)*3:(transf_index+1)*3]
        #Transform the image
        im_transformed = cv.warpPerspective(image,  np.linalg.inv(T_norm) @ h @ T_norm, warp_shape)
        
        return im_transformed

In [3]:
#This function performs the stitching of the given set of images
def stitch_images(imgs, #Images to be stitched
                  graph, #Graph used to define the order of images to be stitched
                  root_idx, #Index of the root of the graph
                  father_idx, #Index of the father of the current image in the graph
                  T_norm, #Normalization matrix
                  Z,  #Matrix containing the homographies between the images
                  warp_shape = [2000,1000], #Shape of the stitched image
                  save_output=True, #If True save the output of the procedure
                  output_dir = "output", #Directory where to save the output
                  verbose=False #If True prints the intermediate results
                 ):
    base_image = cv.warpPerspective(imgs[root_idx], np.eye(3), warp_shape) #Transform the root image (just to get images of the same dimension)
    nb_it = graph.neighbors(root_idx) #Find the neighbours of the current image in the graph
    
    #For each neinghbour...
    for n in nb_it:
        #...that is not the father image
        if n != father_idx:
            #Call again the procedure to stitch images below in the graph
            child = stitch_images(imgs, graph, n, root_idx, T_norm, Z, warp_shape, save_output, output_dir, verbose)
            #Transform the obtained image in the reference frame of the current one
            tr_child = transform_image(root_idx, n, child, T_norm, Z, warp_shape)
            #Stitch images
            base_image =  np.maximum(base_image,tr_child)
    
    #Print intermediate results and save the output if required
    if verbose:
        figure(figsize=(40, 40), dpi=80)
        plt.imshow(base_image,),plt.show() 
    if save_output:
        cv.imwrite(os.path.join(output_dir,f"{root_idx}.jpg"), cv.cvtColor(base_image,cv.COLOR_RGB2BGR))
    return base_image  

In [5]:
#This function can be used to manage the whole procedure of BasicStitching
def basic_stitching(dataset_name, #Name of the dataset to be used
                    imgs, #Images to be stitched
                    T_norm, #Normalization matrix
                    Z, #Matrix containing the homographies between images
                    adj_matrix, #Adjacency matrix of the graph built according to the matches between the images
                    weight_matrix, #Weight matrix of the graph built according to the matches between the images
                    idx_ref = 0, #Index of the reference image
                    verbose = True, #If True print also intermediate results
                    save_output = True, #If True saves the output of the procedure
                    stitching_dir = "stitched", #Name of the father directory
                    partial_results_dir = "partial", #Name of the directory used to store the intermediate results
                    basic_stitching_dir = "basic_stitching", #Name of the directory where to store the final results
                    warp_shape = [10000,10000]
                   ):
    
    #Directory where to store the output
    output_dir = os.path.join(os.path.join(stitching_dir,dataset_name),basic_stitching_dir)
    #Directory where to store the intermediate results
    partial_output_dir = os.path.join(output_dir, partial_results_dir)
    
    #Check whether the directories already exist and create them if not
    if save_output:
        if os.path.isdir(output_dir):   
            shutil.rmtree(output_dir)
        if not os.path.isdir(output_dir):   
            os.makedirs(output_dir)
        if not os.path.isdir(partial_output_dir):
            os.makedirs(partial_output_dir)
    
    #Build the graph starting from the images
    graph = Utils.build_and_print_graph(imgs, 
                                  adj_matrix, 
                                  weight_matrix, 
                                  save_output, 
                                  output_dir,
                                  "graph")
    
    #Compute the maximum SP of the obtained graph and print it
    spanning_tree = nx.maximum_spanning_tree(graph)
    Utils.print_graph(spanning_tree, save_output, output_dir, "spanning-tree")
    
    #Stitch images together
    root = idx_ref
    stitched_image = stitch_images(imgs, 
                                   spanning_tree, 
                                   root, 
                                   root, 
                                   T_norm,
                                   Z, 
                                   warp_shape, 
                                   save_output, 
                                   partial_results_dir, 
                                   verbose)
    
    #If required save the output
    if save_output:
        cv.imwrite(os.path.join(output_dir,"stitched.jpg"), cv.cvtColor(stitched_image,cv.COLOR_RGB2BGR))
        
    #Plot the stitched image
    figure(figsize=(40, 40), dpi=80)
    plt.imshow(stitched_image,),plt.show() 
    
    return stitched_image